In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from io import StringIO

In [2]:
def contains_pattern(row, pattern):
      return any(pattern in str(cell) for cell in row)
def remove_after_colon(s):
      return s.split(':')[0]

def process_df(filename):
  #read and clean
  with open(filename) as f:
    lines = [line for line in f if not line.startswith('##')]

  # Parse the filtered content with pandas
  q = pd.read_csv(StringIO(''.join(lines)),sep="\t")

  #q=pd.read_csv(filename,comment='##',sep="\t")
  q.drop(columns=["#CHROM","ID","QUAL","FILTER","INFO","FORMAT"],inplace=True)
  col_names=[]
  for c in q.columns:
      if c[0]=="/":
        n=c.split("/")[7]
        s=n.split("_")[1]
        col_names.append(s)
      else:
        col_names.append(c)
  q.columns=col_names

  #only SNP and biallelic
  df_filtered = q[q['REF'].str.len() == 1]
  df_filtered = df_filtered[df_filtered['ALT'].str.len() == 1]

  #take out quality info, leave just genotype
  df_filtered['POS'] = df_filtered['POS'].astype(str)
  df_filtered = df_filtered.applymap(remove_after_colon)

  #remove heterozygotes
  pattern = "0/1"
  df_filtered = df_filtered[~df_filtered.apply(lambda row: contains_pattern(row, pattern), axis=1)]

  # Replace "0" with values from "REF" column
  for col in df_filtered.columns[3:]:
    df_filtered[col] = df_filtered.apply(lambda row: row['REF'] if row[col] == '0/0' else row[col], axis=1)
    df_filtered[col] = df_filtered.apply(lambda row: row['ALT'] if row[col] == '1/1' else row[col], axis=1)

  #count precision of marker
  bases=[]
  freq = []
  for index, row in df_filtered.iterrows():
     most_c = Counter(row[3:]).most_common(1)[0] #('G', 15)
     bases.append(most_c[0])
     freq.append(most_c[1]/len(row[3:]))

  df_filtered=df_filtered.iloc[:,:3]
  df_filtered["val"]=bases
  df_filtered["percentage"]=freq

  #keep rows where everyone is the same
  #q=df_filtered[df_filtered[df_filtered.columns[3:]].nunique(axis=1) == 1]

  return df_filtered

In [8]:
#Note: needs subsetted vcfs. Described in Part 2 nucleotide diversity section.

mypath='/gpfs1/home/c/p/cpetak/WGS/Urchin_inversions/genotype_spec_vcfs/'
mychr='NW_022145606.1'
posi='_16610900_16737625_'#_30779143_31460853_'#'_29427741_29776196_'#'_8291260_9094120_' #'_28950395_29566247_' #'_33842803_34582517_' #'_14219351_14298524_' #'_12702886_16793794_' # #

q=process_df(mypath+mychr+posi+'homoq.vcf')
print(q) #most common base and it's frequency

p=process_df(mypath+mychr+posi+'homop.vcf')
print(p) #most common base and it's frequency



/tmp/ipykernel_3728484/992128992.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df_filtered.applymap(remove_after_colon)


             POS REF ALT val  percentage
0          41860   T   A   T         0.6
4          41963   G   A   G         0.8
5          41965   A   T   A         1.0
7          42003   A   C   A         1.0
11         42064   A   T   A         0.8
...          ...  ..  ..  ..         ...
261875  52413291   A   G   A         1.0
261877  52413444   A   T   T         1.0
261879  52413634   C   A   C         1.0
261881  52413937   T   A   T         1.0
261882  52414011   G   T   G         1.0

[80060 rows x 5 columns]


/tmp/ipykernel_3728484/992128992.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df_filtered.applymap(remove_after_colon)


             POS REF ALT val  percentage
152        75883   A   G   G    1.000000
154        76984   T   G   G    0.976744
155        76989   C   A   A    0.941860
158        77466   C   T   T    1.000000
159        77533   C   A   A    1.000000
...          ...  ..  ..  ..         ...
261861  52406145   G   A   A    0.930233
261864  52406293   A   C   C    0.906977
261870  52413021   A   C   C    0.976744
261874  52413278   A   G   G    1.000000
261877  52413444   A   T   T    1.000000

[3152 rows x 5 columns]


In [9]:
#out of all the positions on this chromosome, these are consistenlty different between the orientations.

merged_df = pd.merge(q, p, on='POS', how='inner') #only keep positions that are present in both p and q
nonuniqe=merged_df[merged_df[merged_df.columns[[3,7]]].nunique(axis=1) != 1] #only keep positions that are different between p and q
df = nonuniqe[~nonuniqe.apply(lambda row: contains_pattern(row, "/"), axis=1)]
df = df.drop(['REF_y', 'ALT_y'], axis=1)
df = df.rename(columns={'REF_x': 'REF','ALT_x': 'ALT', 'val_x': 'q', 'val_y': 'p','percentage_x': 'Frequency among q','percentage_y': 'Frequency among p' })

print(df)

df = df[df["Frequency among q"]>0.9]
df = df[df["Frequency among p"]>0.9]

df.to_csv(f'{mypath+mychr+posi}_snp_markers.csv', index=False)

          POS REF ALT  q  Frequency among q  p  Frequency among p
960  16717165   A   G  A                1.0  G           1.000000
961  16718391   G   T  G                1.0  T           0.988372
962  16718762   T   C  T                1.0  C           0.988372
963  16718803   A   G  A                1.0  G           1.000000
964  16718979   A   G  A                1.0  G           0.988372
965  16719243   C   T  C                1.0  T           0.988372
966  16719308   G   A  G                1.0  A           1.000000
967  16719315   G   A  G                1.0  A           1.000000
968  16719316   T   C  T                1.0  C           1.000000
969  16719387   C   G  C                1.0  G           1.000000
970  16719657   A   T  A                1.0  T           1.000000
971  16719680   C   T  C                1.0  T           1.000000
972  16719723   C   G  C                1.0  G           0.988372
973  16719727   A   T  A                1.0  T           0.988372
974  16719

In [ ]:
#Round frequency values

import pandas as pd
import glob
import os

# Find all CSV files that contain 'snp_marker' in the filename
csv_files = glob.glob("*snp_marker*.csv")

for file in csv_files:
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Round all numeric columns to 3 decimal places
    df = df.round(3)
    
    # Save it back (overwrite original)
    df.to_csv(file, index=False)

    print(f"Processed: {file}")


Processed: NW_022145600.1_33842803_34582517_snp_markers.csv
Processed: NW_022145597.1_14219351_14298524_snp_markers.csv
Processed: NW_022145594.1_39429440_42445994_snp_markers.csv
Processed: NW_022145594.1_12702886_16793794_snp_markers.csv
Processed: NW_022145610.1_30779143_31460853_snp_markers.csv
Processed: NW_022145606.1_16610900_16737625_snp_markers.csv
Processed: NW_022145601.1_28950395_29566247_snp_markers.csv
Processed: NW_022145609.1_29427741_29776196_snp_markers.csv
Processed: NW_022145603.1_8291260_9094120_snp_markers.csv
